# Prepare two worker processes

In [1]:
addprocs(2)

2-element Array{Int64,1}:
 2
 3

In [2]:
workers()

2-element Array{Int64,1}:
 2
 3

# Create SharedArray objects

- `SA` holds a 2D shared array with random numbers
- `SR` holds a 2-element shared array that stores summarized computation results

In [4]:
SA = convert(SharedArray{Float64, 2}, rand(6,2))

6×2 SharedArray{Float64,2}:
 0.80396   0.219052 
 0.463357  0.522406 
 0.587913  0.0413235
 0.891794  0.110544 
 0.154502  0.292556 
 0.867073  0.411577 

In [5]:
SR = SharedArray{Float64,1}(2)

2-element SharedArray{Float64,1}:
 0.0
 0.0

# Quick Tests about distributed computation

In [12]:
# using worker 2, compute and store results for first column
# it returns a future object
SR[:] = zeros(2)
display(SR)


2-element SharedArray{Float64,1}:
 0.0
 0.0

In [13]:
@spawnat 2 SR[1] = sum(SA[:,1])

Future(2, 1, 22, Nullable{Any}())

In [14]:
# examine SR.  Hopefully it's already filled in by worker 2.
SR

2-element SharedArray{Float64,1}:
 3.7686
 0.0   

In [15]:
# do the same thing with worker 3.  Again, don't wait for it.
@spawnat 3 SR[2] = sum(SA[:,2])

Future(3, 1, 23, Nullable{Any}())

In [16]:
# hooray!
SR

2-element SharedArray{Float64,1}:
 3.7686 
 1.59746

# Distributed vs Single Process Computation

Each worker will compute the sum of random numbers and accumulate the same value in a for-loop 100mm times.  Ther result is stored in the SR array.

We can see that the distributed computing version is about 2x as fast.

In [17]:
# distributed version
SR[1:2] = 0.0
display(SR)
tic()
f1 = @spawnat 2 SR[1] = begin x = 0.0; for i in 1:100_000_000 x += sum(SA[:,1]); end; x; end;
f2 = @spawnat 3 SR[2] = begin x = 0.0; for i in 1:100_000_000 x += sum(SA[:,2]); end; x; end;
@time wait.([f1, f2])
display(SR)
toc()

2-element SharedArray{Float64,1}:
 0.0
 0.0

2-element SharedArray{Float64,1}:
 3.7686e8 
 1.59746e8

 17.341014 seconds (41.24 k allocations: 2.131 MiB, 0.40% gc time)
elapsed time: 17.812878185 seconds


17.812878185

In [18]:
# single-process version
SR[1:2] = 0.0
display(SR)
tic()
SR[1] = begin x = 0.0; for i in 1:100_000_000 x += sum(SA[:,1]); end; x; end;
SR[2] = begin x = 0.0; for i in 1:100_000_000 x += sum(SA[:,2]); end; x; end;
display(SR)
toc()

2-element SharedArray{Float64,1}:
 0.0
 0.0

2-element SharedArray{Float64,1}:
 3.7686e8 
 1.59746e8

elapsed time: 33.746553001 seconds


33.746553001

# Clean up

Shutting down the compute worker nodes.

In [19]:
rmprocs(workers())

Task (done) @0x0000000119976410

In [20]:
procs()

1-element Array{Int64,1}:
 1